In [1]:
import pandas as pd
import os
import sys
print(os.getcwd())
sys.path.append('/Users/calvindu/School/Projects/FINANCE_TRACKER/')
import joblib
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from src.plugins.categorization_model.dsci_src.feature_engineering.numerical_features import (
    add_numerical_features,
)
from src.plugins.categorization_model.dsci_src.feature_engineering.text_features import (
    add_text_features,
)
from src.plugins.categorization_model.dsci_src.feature_engineering.text_features import (
    text_cleanup,
)
from src.plugins.categorization_model.dsci_src.feature_engineering.time_features import (
    add_time_features,
)
from src.plugins.categorization_model.dsci_src.feature_engineering.transaction_matching_features import (
    count_transactions_matching_feature,
)
from src.plugins.categorization_model.dsci_src.feature_engineering.transaction_matching_features import (
    transaction_matching,
)
from src.plugins.categorization_model.dsci_src import model

/Users/calvindu/School/Projects/FINANCE_TRACKER/src/plugins/categorization_model/dsci_src


In [5]:
data = pd.read_csv("/Users/calvindu/School/Projects/FINANCE_TRACKER/src/plugins/categorization_model/categorized_data/categorized.csv")
model.create_model(data)

['amount', 'matched_transactions_count', 'day_of_week', 'month_of_year', 'description_length', 'num_tokens']
['account', 'amount_category', 'amount_sign']
Accuracy: 0.626984126984127


/Users/calvindu/School/Projects/FINANCE_TRACKER/src/plugins/categorization_model/dsci_src/model.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y = df["category"]


In [3]:
def use_model(df: pd.DataFrame) -> pd.DataFrame:
    df = df[['account', 'date', 'description', 'amount']]
    df, preprocessor = model.create_preprocessor(df)
    X_preprocessed = preprocessor.fit_transform(df)

    loaded_knn_model = joblib.load(
        "/Users/calvindu/School/Projects/FINANCE_TRACKER/src/plugins/categorization_model/dsci_src/knn_model.joblib",
    )

    predicted_category = loaded_knn_model.predict(X_preprocessed)

    df["category"] = predicted_category

    return df

use_model(data)

['amount', 'matched_transactions_count', 'day_of_week', 'month_of_year', 'description_length', 'num_tokens']
['account', 'amount_category', 'amount_sign']


/Users/calvindu/School/Projects/FINANCE_TRACKER/src/plugins/categorization_model/dsci_src/feature_engineering/transaction_matching_features.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_matched_column] = [[] for _ in range(len(df))]
/Users/calvindu/School/Projects/FINANCE_TRACKER/src/plugins/categorization_model/dsci_src/feature_engineering/transaction_matching_features.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{matched_column}_count"] = df[matched_column].apply(len)
/Users/cal

,account,date,description,amount,matched_transactions,matched_transactions_count,day_of_week,month_of_year,amount_sign,amount_category,cleaned_description,description_length,num_tokens,category
0,BMO_CHECKING,2023-07-13,NEW HONG FATT B,28.00,[],0,3,7,positive,small_positive,new hong fatt b,15,4,food_drink
1,BMO_CHECKING,2023-07-13,NEW HONG FATT B,28.00,[],0,3,7,positive,small_positive,new hong fatt b,15,4,food_drink
2,BMO_CHECKING,2023-07-12,TF 0005191230208404641,101.02,[],0,2,7,positive,positive,tf 0005191230208404641,23,2,etransfer_out
3,BMO_CHECKING,2023-07-12,TF 0005191230208404641,101.02,[],0,2,7,positive,positive,tf 0005191230208404641,23,2,etransfer_out
4,AMEX,2023-07-11,TIM HORTONS #2387 TORONTO,10.15,[],0,1,7,positive,small_positive,tim hortons 2387 toronto,24,4,etransfer_in
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3200,CIBC_CHECKING,2020-05-11,Internet Banking INTERNET TRANSFER 000000284575,-30.00,[],0,0,5,negative,small_negative,internet banking internet transfer 000000284575,47,5,food_drink
3201,CIBC_CHECKING,2020-05-08,Point of Sale - Interac RETAIL PURCHASE 012913...,5.54,[],0,4,5,positive,small_positive,point of sale interac retail purchase 01291313...,65,10,etransfer_in
3202,CIBC_CHECKING,2020-05-07,Internet Banking E-TRANSFER 102453365575 CHARV...,-30.00,[],0,3,5,negative,small_negative,internet banking e transfer 102453365575 charv...,52,7,food_drink
3203,CIBC_CHECKING,2020-04-30,Branch Transaction SERVICE CHARGE DISCOUNT,-4.95,[3204],1,3,4,negative,small_negative,branch transaction service charge discount,42,5,credit_card_payment
